In [ ]:
# create some tables in database system for sql practice 

In [1]:
# import necessary lybraries

import pandas as pd
import numpy as np


# import connection 

import sqlite3
conn = sqlite3.connect('pokemon.db')

cur  = conn.cursor()

In [7]:
cur.execute(""" create table emp(empno integer primary key,
ename text not null, job text,mgr integer,
hiredate numeric,sal integer,comm integer,deptno integer not null);""")


In [18]:
cur.execute(""" create table dept(deptno integer primary key,
dname text not null, loc text);""")

In [22]:
cur.execute("""select * from dept;""")
cur.fetchall()

[(10, 'ACCOUNTING', 'NEW YORK'),
 (20, 'RESEARCH', 'DALLAS'),
 (30, 'SALES', 'CHICAGO'),
 (40, 'OPERATIONS', 'BOSTON')]

In [30]:
cur.execute("select * from emp;")
cur.fetchall()

[(7369, 'SMITH', 'CLERK', 7902, '17-DEC-80', 800, None, 20),
 (7499, 'ALLEN', 'SALESMAN', 7698, '20-FEB-81', 1600, 300, 30),
 (7521, 'WARD', 'SALESMAN', 7698, '22-FEB-81', 1250, 500, 30),
 (7566, 'JONES', 'MANAGER', 7839, '2-APR-81', 2975, None, 20),
 (7654, 'MARTIN', 'SALESMAN', 7839, '2-APR-81', 1250, 1400, 30),
 (7902, 'FORD', 'ANALYST', 7566, '3-DEC-81', 3000, None, 20)]

In [28]:
cur.execute(""" update emp 
set ename = 'FORD'
where empno = 7902""")

In [29]:
cur.execute(""" update emp 
set deptno = 30
where empno = 7499""")

In [13]:
# insert values

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,deptno) 
values(7369,'SMITH','CLERK',7902,'17-DEC-80',800,20);""")

IntegrityError: UNIQUE constraint failed: emp.empno

In [14]:
cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,comm,deptno) 
values(7499,'ALLEN','SALESMAN',7698,'20-FEB-81',1600,300,20);""")

In [16]:
cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,comm,deptno) 
values(7521,'WARD','SALESMAN',7698,'22-FEB-81',1250,500,30);""");

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,comm,deptno) 
values(7654,'MARTIN','SALESMAN',7839,'2-APR-81',1250,1400,30);""")

In [15]:
cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,deptno) 
values(7566,'JONES','MANAGER',7839,'2-APR-81',2975,20);""");

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,deptno) 
values(7902,'ford','ANALYST',7566,'3-DEC-81',3000,20);""")

In [21]:
# insert into dept table

cur.execute(""" insert into dept(deptno,dname,loc) values(10,'ACCOUNTING','NEW YORK')""");
cur.execute(""" insert into dept(deptno,dname,loc) values(20,'RESEARCH','DALLAS')""");
cur.execute(""" insert into dept(deptno,dname,loc) values(30,'SALES','CHICAGO')""");
cur.execute(""" insert into dept(deptno,dname,loc) values(40,'OPERATIONS','BOSTON')""");

In [62]:
# practice sub queries on emp and dept table 


cur.execute(""" SELECT * FROM emp 
WHERE mgr = (SELECT mgr FROM emp WHERE ename = 'JONES');
""")

df_sql1 = pd.DataFrame(cur.fetchall(),columns =[ item[0] for item in cur.description ])
df_sql1

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7566,JONES,MANAGER,7839,2-APR-81,2975,NaN,20
1,7654,MARTIN,SALESMAN,7839,2-APR-81,1250,1400.0,30


In [61]:
# pairwise comparison

cur.execute(""" SELECT * FROM emp 
WHERE (deptno, mgr) = (SELECT deptno, mgr FROM emp WHERE ename = 'FORD');
""")

df_sql2 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql2

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7902,FORD,ANALYST,7566,3-DEC-81,3000,None,20


In [60]:
# non pairwise comparison

cur.execute(""" SELECT * FROM emp 
WHERE deptno = (SELECT deptno FROM emp WHERE ename='FORD') 
AND mgr = (SELECT mgr FROM emp WHERE ename='FORD')
""")

df_sql3 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql3

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7902,FORD,ANALYST,7566,3-DEC-81,3000,None,20


In [52]:
# multi row sub query example 

cur.execute(""" SELECT * FROM emp 
WHERE deptno IN (SELECT deptno FROM emp WHERE job='CLERK') 
AND job <> 'CLERK'; """)

df_sql4 = pd.DataFrame(cur.fetchall(),columns = columns )
df_sql4

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7566,JONES,MANAGER,7839,2-APR-81,2975,None,20
1,7902,FORD,ANALYST,7566,3-DEC-81,3000,None,20


In [58]:
# Scaler Sub Query example

cur.execute(""" SELECT  deptno FROM emp WHERE ename='FORD'; """)

df_sql5 = pd.DataFrame(cur.fetchall(),columns = [item[0] for item in cur.description] )
df_sql5

,deptno
0,20


In [57]:
cur.description

(('deptno', None, None, None, None, None, None),)

In [47]:
columns = [ item[0] for item in cur.description ]
columns

['empno', 'ename', 'job', 'mgr', 'hiredate', 'sal', 'comm', 'deptno']

In [59]:
# Inline View Sub Query example

cur.execute(""" SELECT * FROM (SELECT * FROM emp ORDER BY sal DESC); """)

df_sql6 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql6

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7902,FORD,ANALYST,7566,3-DEC-81,3000,NaN,20
1,7566,JONES,MANAGER,7839,2-APR-81,2975,NaN,20
2,7499,ALLEN,SALESMAN,7698,20-FEB-81,1600,300.0,30
3,7521,WARD,SALESMAN,7698,22-FEB-81,1250,500.0,30
4,7654,MARTIN,SALESMAN,7839,2-APR-81,1250,1400.0,30
5,7369,SMITH,CLERK,7902,17-DEC-80,800,NaN,20


In [68]:
# Co-related sub query examples

cur.execute(""" SELECT e.ename ,e.deptno, (SELECT d.dname 
FROM dept as d WHERE d.deptno = e.deptno) department FROM emp as e ; """)

df_sql7 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql7

OperationalError: no such column: e.deptno